In [1]:
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from model import Tudui
from torch.utils.tensorboard import SummaryWriter
import time

In [2]:
train_data = torchvision.datasets.CIFAR10(root='./CIFAR', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_data = torchvision.datasets.CIFAR10(root='./CIFAR', train=False, download=True, transform=torchvision.transforms.ToTensor())

train_data_size = len(train_data)
test_data_size = len(test_data)

print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))

Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为：50000
测试数据集的长度为：10000


In [3]:
train_data_loader = DataLoader(train_data, batch_size=64)
test_data_loader = DataLoader(test_data, batch_size=64)

In [4]:
tudui = Tudui()
tudui = tudui.cuda()
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.cuda()
learning_rate = 1e-2
optimizer = torch.optim.SGD(tudui.parameters(), lr=learning_rate)

In [5]:
total_train_step = 0
total_test_step = 0
epoch = 10

writer = SummaryWriter('logs')

start_time = time.time()

for i in range(epoch):
    print("-----第{}轮训练开始-----".format(i+1))

    tudui.train()
    for data in train_data_loader:
        imgs, targets = data
        imgs = imgs.cuda()
        targets = targets.cuda()
        outputs = tudui(imgs)
        loss = loss_fn(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_step += 1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print("100次训练用时：{}".format(end_time-start_time))
            print("训练次数：{}, Loss:{}".format(total_train_step, loss.item()))
            writer.add_scalar('train_loss', loss.item(), total_train_step)

    total_test_loss = 0
    total_accuracy = 0

    tudui.eval()
    with torch.no_grad():
        for data in test_data_loader:
            imgs, targets = data
            imgs = imgs.cuda()
            targets = targets.cuda()
            outputs = tudui(imgs)
            loss = loss_fn(outputs, targets)
            total_accuracy += (outputs.argmax(1) == targets).sum().item()
            total_test_loss += loss.item()

    print("第{}轮训练的Loss为：{}".format(i+1, total_test_loss))
    print("第{}轮训练的准确率为：{}".format(i+1, total_accuracy/test_data_size))
    writer.add_scalar('test_loss', total_test_loss, total_test_step)
    writer.add_scalar('accuracy', total_accuracy/test_data_size, total_test_step)
    total_test_step += 1

    torch.save(tudui, './checkpoints/tudui_{}.pth'.format(i+1))
    print("-----第{}轮训练结束，模型已保存-----".format(i+1))

-----第1轮训练开始-----
100次训练用时：0.43697214126586914
训练次数：100, Loss:2.2904109954833984
100次训练用时：0.7653307914733887
训练次数：200, Loss:2.2753400802612305
100次训练用时：1.083570957183838
训练次数：300, Loss:2.2487893104553223
100次训练用时：1.418731927871704
训练次数：400, Loss:2.148871421813965
100次训练用时：1.7559847831726074
训练次数：500, Loss:2.065931797027588
100次训练用时：2.084712505340576
训练次数：600, Loss:2.0233144760131836
100次训练用时：2.4225716590881348
训练次数：700, Loss:2.019643783569336
第1轮训练的Loss为：315.4077181816101
第1轮训练的准确率为：0.2694
-----第1轮训练结束，模型已保存-----
-----第2轮训练开始-----
100次训练用时：3.194183349609375
训练次数：800, Loss:1.8995676040649414
100次训练用时：3.53100323677063
训练次数：900, Loss:1.88705313205719
100次训练用时：3.867772340774536
训练次数：1000, Loss:1.9513185024261475
100次训练用时：4.220137119293213
训练次数：1100, Loss:1.9499115943908691
100次训练用时：4.555300712585449
训练次数：1200, Loss:1.7303991317749023
100次训练用时：4.90251350402832
训练次数：1300, Loss:1.714880347251892
100次训练用时：5.240357398986816
训练次数：1400, Loss:1.775428295135498
100次训练用时：5.591324329376221
训练次数：1500,